In [ ]:
!pip install -U transformers
!pip install geopy
!pip install fugashi[unidic-lite]
!pip install ipadic
!pip install soyclustering

In [ ]:
import os
import torch
import pandas as pd
import numpy as np
import math
from geopy.distance import geodesic
from google.colab import drive
from transformers import AutoModel, AutoTokenizer, Trainer, TrainingArguments, BertTokenizerFast, PreTrainedTokenizerFast, EncoderDecoderModel
from scipy.sparse import csr_matrix

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
WorkDir = 'drive/MyDrive/Colab Notebooks/이지은님/일본 에어비앤비 프로젝트'

In [ ]:
data_path = os.path.join(WorkDir, 'data')

In [ ]:
airbnb_data = pd.read_csv(os.path.join(data_path, 'airbnb_data_sample.csv'))
oshima_data = pd.read_csv(os.path.join(data_path, 'oshimaland_dataset_final.csv'))
open_data = pd.read_csv(os.path.join(data_path, 'open_data_sample.csv'))

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

----

### 일본어 -> 한국어 번역

In [ ]:
def translate(text_src):
    embeddings = src_tokenizer(text_src, return_attention_mask=False, return_token_type_ids=False, return_tensors='pt')
    embeddings = {k: v for k, v in embeddings.items()}
    output = model.generate(**embeddings)[0, 1:-1]
    text_trg = trg_tokenizer.decode(output.cpu())
    return text_trg

In [ ]:
encoder_model_name = "cl-tohoku/bert-base-japanese-v2"
decoder_model_name = "skt/kogpt2-base-v2"

src_tokenizer = BertTokenizerFast.from_pretrained(encoder_model_name)
trg_tokenizer = PreTrainedTokenizerFast.from_pretrained(decoder_model_name)
model = EncoderDecoderModel.from_pretrained("figuringoutmine/translator-for-travel-jp-to-kr").to(device)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'BertTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'BertTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
translated = list()
with torch.no_grad():
    for info in oshima_data['info'].values:
        encoded = src_tokenizer(info, return_attention_mask=False, return_token_type_ids=False, return_tensors='pt')
        encoded = {k:v.to(device) for k, v in encoded.items()}
        output = model.generate(**encoded)[0, 1:-1]
        translated.append(trg_tokenizer.decode(output.cpu()))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
oshima_data['translated'] = translated

### Bert Embedding Base Clustering

In [ ]:
from soyclustering import SphericalKMeans

In [ ]:
model = AutoModel.from_pretrained('cl-tohoku/bert-base-japanese')
tokenizer = AutoTokenizer.from_pretrained('cl-tohoku/bert-base-japanese')

In [ ]:
embeddings = list()
with torch.no_grad():
    for info in oshima_data['info'].values:
        encoded = tokenizer(info, return_tensors='pt')
        output = model(**encoded)
        embedding = output.pooler_output[0].detach().cpu().numpy().tolist()
        embeddings.append(embedding)
embeddings = csr_matrix(embeddings)

In [ ]:
spherical_kmeans = SphericalKMeans(
    n_clusters=5,    #  5개의 군집으로 군집화
    max_iter=100,    # 최대 Iteration 수
    verbose=1,
    init='similar_cut'
)

In [ ]:
labels = spherical_kmeans.fit_predict(embeddings)

initialization_time=0.026657 sec, sparsity=0.999
n_iter=1, changed=493, inertia=102.196, iter_time=0.625 sec, sparsity=1.0
n_iter=2, changed=99, inertia=64.359, iter_time=0.621 sec, sparsity=1.0
n_iter=3, changed=34, inertia=61.347, iter_time=0.614 sec, sparsity=1.0
n_iter=4, changed=29, inertia=59.937, iter_time=0.712 sec, sparsity=1.0
n_iter=5, changed=10, inertia=59.460, iter_time=1.006 sec, sparsity=1.0
n_iter=6, changed=15, inertia=59.382, iter_time=0.864 sec, sparsity=1.0
n_iter=7, changed=16, inertia=59.229, iter_time=0.604 sec, sparsity=1.0
n_iter=8, changed=7, inertia=59.064, iter_time=1.251 sec, sparsity=1.0
n_iter=9, changed=9, inertia=59.034, iter_time=1.719 sec, sparsity=1.0
n_iter=10, changed=2, inertia=58.966, iter_time=1.580 sec, sparsity=1.0
n_iter=11, changed=1, inertia=58.962, iter_time=1.306 sec, sparsity=1.0
Early converged.


In [ ]:
oshima_kmeans = oshima_data.copy()
oshima_kmeans['labels'] = labels

In [ ]:
oshima_kmeans[oshima_kmeans['labels']==0].sample(10)    # 투신자살..?

,info,address,occurred_date,recorded_date,translated,labels
465,部屋番号不明。老人の遺体発見。緊急車両と警官の出動を、向かいにある予備校の学生多数が目撃。,東京都町田市森野一丁目3214アールヴェール町田森野,2017年夏頃,平成30年8月6日,방호수 불명. 노인들의 시신 발견. 긴급 차량과 경찰의 출동을,0
285,マンションの住民ではないと思われる身元不明の方が深夜に飛び降り自殺\n明け方に発見された,兵庫県神戸市中央区御幸通六丁目130,令和4年11月3日,令和5年1月15日,아파트 주민이 아닌 신원이 밝혀지지 않은 신원이 심야에 뛰어내려가 자살한 것으로 밝혀,0
439,ホテルコンチネンタル府中、地下１階\n自己啓発主催者ら書類送検＝セミナー中に男性死亡―遺棄致...,東京都府中市府中町一丁目51,平成22年1月27日,平成27年8月17日,"호텔 콘티넨탈 부중, 지하 2층 자기계발 주최자 등 서류송부",0
497,告知事項あり（不動産情報より）,東京都東村山市栄町三丁目128 久米川マンション 5階,2020年2月現在,令和2年2月14日,공지사항 있습니다~ 부동산 정보부터!,0
258,ラウンドワン三宮店の女子トイレで産まれたばかりの胎児が放棄されてた事件がありました。,兵庫県神戸市中央区小野柄通六丁目1−１７,7年くらい前だったと思う,平成27年2月18日,라운드 원미야마(ラ운드 원미야) 점 화장실에서 갓 태어난 태,0
482,29歳女性が首吊り自殺,東京都三鷹市上連雀九丁目1414,2018年3月9日,平成30年3月16日,29세 여성이 목도리 자살,0
290,アルティス新金岡\n2000年に建築されたマンションですが、2000年〜2005年の間？に飛...,大阪府堺市北区新金岡町5丁15,平成29年1月6日,平成29年9月3日,알티스카 신카네카 2000년에 지어진 아파트인데 2000년~2005년 사이에 1층으...,0
302,※告知事項有（入居者死亡）,東京都杉並区阿佐谷南一丁目273 本間アパート A7号室,2021,令和3年6月9日,*공지사항 없음,0
197,屋上より女性が飛び降り自殺,東京都世田谷区世田谷三丁目2320,平成29年1月18日,平成29年11月12日,옥상에서 여자가 뛰어내리고 자살,0
60,早朝に屋上から若い女性の飛び降り自殺。,東京都新宿区歌舞伎町一丁目126 歌舞伎町ビル,平成21年6月29日,平成27年6月15日,이른 아침에 옥상에서 젊은 여자의 뛰어내리기 자살.,0


In [ ]:
oshima_kmeans[oshima_kmeans['labels']==1].sample(10)    # 자살

,info,address,occurred_date,recorded_date,translated,labels
462,402号室 初老の男性 餓死,東京都町田市森野五丁目134,2013年,平成30年3月12日,402호실 초로의 남성 폭사,1
560,都営武蔵野アパート敷地内\n30歳代くらいの男性遺体発見\n死因は全身打撲による外傷性ショック死,東京都武蔵野市緑町二丁目64,平成17年12月28日,平成25年11月11日,도영구 무라타미 아파트 부지 내 30대 정도의 남성 시신 발견사,1
210,10階 角部屋\n告知事項あり,東京都大田区蒲田五丁目4810 ハイライフ蒲田,令和4年10月16日,令和4年10月17日,10층 각 방 고지사항 있음.,1
558,心理的瑕疵物件\n不動産情報に記載,東京都練馬区関町北三丁目45 アイル関町 2階,時期不明,平成29年10月27日,심리적 하자 금전 부동산 정보에 기재,1
537,一階角部屋自殺,東京都羽村市栄町二丁目118,2018年夏ごろ,平成30年10月8日,1층 각방 자살,1
567,6階 角部屋\n告知事項有り,東京都国分寺市戸倉一丁目825 PlaceK,令和4年10月6日,令和4年10月7日,6층 각방고시사항 있음,1
116,車両事故死,東京都新宿区歌舞伎町二丁目917,平成29年8月19日,平成29年8月19日,차량사고,1
501,独居高齢者の不審死（事件と事故双方の可能性）,東京都東村山市本町一丁目321,2021年夏,令和4年4月5日,독거노인의 수상한 심사가 일어난 사건 2건과 사고 양측 모두 가능할 것 같,1
325,硫化水素自殺,東京都文京区本郷一丁目332水道橋グランドホテル523,平成22年3月26日,NaN,유황수소 자살,1
566,5階 角部屋\n告知事項有り,東京都国分寺市戸倉一丁目825 PlaceK,令和4年10月6日,令和4年10月7日,5층 각방고시사항 있음,1


In [ ]:
oshima_kmeans[oshima_kmeans['labels']==2].sample(10)    # 사고사 / 고독사

,info,address,occurred_date,recorded_date,translated,labels
49,309号室、709号室、808号室\n告知事項あり,東京都新宿区歌舞伎町二丁目185 プラーズタワー東新宿,令和4年8月20日,令和4年8月20日,"309호실, 709호실, 808호실 공지사항 있으십",2
280,簡易宿泊施設「磯上荘」\n火災により宿泊客が死亡,兵庫県神戸市中央区磯上通二丁目232,平成24年1月25日,平成24年1月25日,간이 숙박시설 '조이상장' 화재로 인해 투숙객이 사망.,2
561,【青酸カリによる死者が出たそうです】\n4号棟の自治会役員の方の証言で、団地内で青酸カリによ...,東京都武蔵野市緑町二丁目34都営武蔵野緑町二丁目第2アパート4号棟,不明,令和2年4月22日,[청산칼로리] 4호동의 시아버지인 분의 증언으로 단지 지구,2
336,一戸建て住宅1人暮らしで男性が亡くなり周辺住民が腐乱臭に気付き通報。その後改装してアパートに...,東京都荒川区荒川四丁目324 サンハイツ,平成17年9月1日,平成30年11月9日,단독주택 단독주택으로 남자가 죽으면서 주변 주민들이 부패 냄새를 알아보고 신고했다.,2
216,本籍・住所・氏名・年齢不詳、男性、身長160センチメートル、中肉\n上記の者は令和2年7月2...,東京都中野区野方一丁目517,令和2年7月28日,令和3年8月11日,"본인이나 주소, 생년월일, 연령, 신장 미스터, 신장 160센티미터, 내용물은 레이와",2
287,看護士から虐待を受けた老人が死亡,大阪府堺市北区新金岡町5丁13病院内,平成21年9月15日,令和元年7月27日,간호사로부터 학대를 받은 노인이 사망,2
21,2019年10月下旬、40代後半男性が部屋の中で孤高死。\n2019年12月下旬に発見される...,東京都新宿区西新宿七丁目1514FONTAINE三須102号室,2019年10月下旬,令和2年2月3日,"2019년 10월 하순, 40대 후반 남성이 방에서 고독사.2019년 12월 하순",2
8,2F 男性死亡 心霊現象あり,東京都新宿区歌舞伎町二丁目4514,平成30年頃,平成31年3月18日,2F 남성 사망 심령 현상,2
231,2階 角部屋 心理的瑕疵あり 2018年6月内装一部リフォーム済み,東京都中野区中野三丁目2321 コーポ池田,平成30年4月22日,平成30年11月2日,2층 가방적용적하적 있어 2018년 6월 인테리어 일부 리모델링 완료,2
493,２２２号室は告知事項あり、室内全体の補修工事が行われたらしいが、どのようなことがあったのか不...,東京都東村山市本町一丁目1324,令和2年2月1日,令和2年5月31日,1호실은 공지사항이며 실내 전체 보수공사가 이루어졌다고 하는데 어떤 일이 있었는지 알 수,2


In [ ]:
oshima_kmeans[oshima_kmeans['labels']==3].sample(10)    # 화재 / 투신 자살

,info,address,occurred_date,recorded_date,translated,labels
234,首吊り自殺,東京都中野区中野三丁目5013,平成23年3月8日,令和3年4月18日,목살 자살,3
170,飛び降りて死亡,東京都墨田区吾妻橋一丁目2330 URリバーピア吾妻橋ライフタワー1907↓,平成30年10月4日,平成30年10月4日,뛰어내려서 사망,3
262,火災による死亡,兵庫県神戸市中央区旭通二丁目1024 2階,令和4年7月28日,令和4年7月29日,화재로 인한 사망,3
341,火災による死亡,東京都荒川区荒川三丁目304,平成21年12月24日,NaN,화재로 인한 사망,3
79,ホテルパシャ702号室にて20～30代の男性が死亡しているのを発見。官報より,東京都新宿区歌舞伎町二丁目1012,令和3年7月13日,令和3年9月22日,호텔파샤 722호실에서 70대 남성이 사망하는 것을 발견.,3
34,甲新ビル9階 23時頃\n店舗利用客が窓から飛び降り即死,東京都新宿区西新宿一丁目185甲新ビル中沖ビル,平成30年8月1日,平成30年9月12日,갑오신빌딩 9층 23시쯤 매점 이용객이 창문으로 뛰어내리며 즉,3
544,火災による死亡,東京都昭島市田中町一丁目349,令和2年9月7日,令和2年9月7日,화재로 인한 사망,3
318,14階非常階段より 飛び降り自殺,東京都文京区本郷一丁目3515 都営本郷一丁目アパート,平成22年7月27日,令和4年10月13日,14층 비상계단보다 뛰어내리고 자살,3
469,熱中症により孤高死,東京都青梅市東青梅二丁目1612,2014年か2015年の8月頃,平成31年1月8日,열사병에 의해 고비고사,3
324,805室 女性飛び降り自殺。,東京都文京区本郷一丁目333,平成26年10月16日,平成26年10月16日,805실 여성 추락 자살.,3


In [ ]:
oshima_kmeans[oshima_kmeans['labels']==4].sample(10)    # 칼...?

,info,address,occurred_date,recorded_date,translated,labels
132,男性飛び降り自殺,東京都新宿区新宿三丁目181,平成18年9月4日,平成26年1月24日,남자애들 뛰어내리 자살,4
184,飛び降り自殺,東京都渋谷区宇田川町103,平成27年8月21日,平成27年8月22日,뛰어내리 자살,4
238,転落死,東京都中野区中野三丁目3012マンション中野ソレイユ9F↓,平成23年1月9日,NaN,회전사,4
519,男性遺体発見,東京都西東京市南町三丁目312,平成19年2月2日,平成26年1月23日,남성 시체 발견,4
534,死体発見,東京都小平市小川町二丁目135631みゆき荘2階,約2年前,平成25年4月17日,시체 발견,4
437,刺殺,東京都府中市宮西町一丁目5,平成19年2月14日,令和3年10月16日,사시미,4
191,自殺,愛知県名古屋市港区港楽二丁目1118,令和2年11月3日,令和3年1月3日,자살,4
552,告知事項ありの記載,東京都清瀬市中清戸四丁目9092 グランディール 1階,不明,平成29年10月24日,공고사항 있음 기재,4
271,転落死,兵庫県神戸市中央区旭通四丁目13シティタワー神戸三宮 上階↓32階ベランダ,令和4年12月22日,令和4年12月23日,회전사,4
316,女性の孤高死,東京都文京区本郷四丁目305,2013年,令和4年2月28日,여성의 고독사,4


----

### 각 숙소별 반경 nKm 내 사고 발생 건수

In [ ]:
def get_distances(locs_1, locs_2):
    n_rows_1 = locs_1.shape[0]
    n_rows_2 = locs_2.shape[0]
    dists = np.empty((n_rows_1, n_rows_2))
    # The loops here are inefficient
    for i in range(n_rows_1):
        for j in range(n_rows_2):
            dists[i, j] = geodesic(locs_1[i], locs_2[j]).km
    return dists

In [ ]:
oshima_with_location = pd.read_csv(os.path.join(data_path, 'oshima_data_sample.csv'))

In [ ]:
crime_locations = oshima_with_location[['latitude', 'longitude']].values

In [ ]:
# 에어비앤비 데이터 사용하여 사고 건수 계산
airbnb_locations = airbnb_data[['latitude', 'longitude']].values

In [ ]:
distance_matrix = get_distances(airbnb_locations, crime_locations)

In [ ]:
n_km = 3    # 3km 이내 범죄 발생 건수

In [ ]:
n_crimes = np.sum((distance_matrix < n_km), axis=1)

In [ ]:
airbnb_data['n_crimes'] = n_crimes

In [ ]:
airbnb_data

,Unnamed: 0,neighbourhood_cleansed,price,room_type,accommodates,bathrooms_text,bedrooms,beds,review_scores_rating,latitude,longitude,full name,n_crimes
0,0,Sumida Ku,"$11,000.00",Entire home/apt,2,1 bath,1.0,2.0,4.78,35.717070,139.826080,東京都 墨田区 八広 二丁目,0
1,1,Kita Ku,"$7,208.00",Private room,1,1 shared bath,NaN,1.0,4.98,35.738440,139.769170,東京都 北区 田端新町 一丁目,0
2,2,Shibuya Ku,"$23,066.00",Entire home/apt,6,1 bath,2.0,4.0,4.76,35.678780,139.678470,東京都 渋谷区 幡ケ谷 二丁目,15
3,3,Setagaya Ku,"$16,000.00",Private room,2,1 shared bath,NaN,2.0,4.94,35.658000,139.671340,東京都 世田谷区 代沢 二丁目,0
4,4,Adachi Ku,"$10,000.00",Private room,4,1.5 shared baths,NaN,NaN,4.71,35.744731,139.797384,東京都 足立区 千住河原町,0
5,5,Shibuya Ku,"$54,782.00",Entire home/apt,3,1 bath,1.0,1.0,4.96,35.651110,139.721650,東京都 渋谷区 広尾 五丁目,0
6,6,Setagaya Ku,"$7,571.00",Private room,1,1 shared bath,NaN,2.0,4.79,35.607420,139.676160,東京都 世田谷区 奥沢 二丁目,0
7,7,Katsushika Ku,"$16,000.00",Entire home/apt,4,2 baths,2.0,4.0,4.93,35.739870,139.851760,東京都 葛飾区 立石 七丁目,0
8,8,Shinjuku Ku,"$39,143.00",Entire home/apt,2,1 bath,1.0,1.0,4.91,35.702630,139.741470,東京都 新宿区 津久戸町,0
9,9,Meguro Ku,"$29,286.00",Entire home/apt,10,2.5 baths,2.0,2.0,4.77,35.620390,139.699830,東京都 目黒区 目黒本町 五丁目,0
